In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
from NoiseKD import Teacher

First hurdle. the embedding layer.  It requires the ints as inputs, not one hot.  i need one hot.  i will also need to rewire the LLM i use to do the same.  hmm.  save the weights, load it into a model that has the same shapes, but accepts one hot arrays, not vocab_indexes.

In [3]:
class TransformerEncoder(nn.Module):
    def __init__(self, embedding_dim, num_heads, hidden_dim, num_layers, dropout):
        super(TransformerEncoder, self).__init__()
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(embedding_dim, num_heads, hidden_dim, dropout),
            num_layers
        )

    def forward(self, x):
        return self.transformer(x)

class SimpleLanguageModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, class_num, num_heads, hidden_dim, num_layers, dropout):
        super(SimpleLanguageModel, self).__init__()

        # Define the embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # Define the transformer encoder
        self.transformer_encoder = TransformerEncoder(embedding_dim, num_heads, hidden_dim, num_layers, dropout)

        # Define the output layer
        self.output_layer = nn.Linear(embedding_dim, class_num)

    def forward(self, input_data):
        # Input_data is of shape (batch_size, sequence_length)
        # Apply embedding layer
        #print(input_data.shape)
        embedded = self.embedding(input_data)
        #print(embedded.shape)
        # Pass through the transformer encoder
        transformed = self.transformer_encoder(embedded)

        # Average along the sequence dimension
        avg_transformed = torch.mean(transformed, dim=1)

        # Apply the output layer
        output = self.output_layer(avg_transformed)

        return torch.sigmoid(output)
    
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

In [4]:
embedding_dim = 16
num_heads = 8
hidden_dim  = 11
num_layers = 2
dropout = 0.1
vocab_size = 80
class_num = 14
batch_size = 23
sequence_length = 160

In [5]:
SLM = SimpleLanguageModel(vocab_size, embedding_dim, class_num, num_heads, hidden_dim, num_layers, dropout)

In [6]:
SLM

SimpleLanguageModel(
  (embedding): Embedding(80, 16)
  (transformer_encoder): TransformerEncoder(
    (transformer): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=16, out_features=16, bias=True)
          )
          (linear1): Linear(in_features=16, out_features=11, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=11, out_features=16, bias=True)
          (norm1): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
        (1): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=16, out_features=16, bias=True)
       

In [7]:
input_data = torch.randint(low=0, high=vocab_size, size=(batch_size, sequence_length))
input_data.shape,SLM(input_data).shape

(torch.Size([23, 160]), torch.Size([23, 14]))

In [8]:
count_parameters(SLM)

4580

In [9]:
teacher_slm = Teacher(SLM,(sequence_length,)) #don't specify batch!!

In [10]:
teacher_slm.configure(dist_type = "ints", gen_m = vocab_size, gen_n = 1000, gen_epochs = 100,gen_lr = 0.005, out_type = "one-hot" ) #this is dying.  might be time for colab!!

Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)
lets try ints!


Configuring Teacher:: 100%|███████████████████| 100/100 [01:17<00:00,  1.29it/s]

Teacher Configured


In [ ]:
#this has a batch dim, but it isn't being ran in batches.  i think i need to do an append to self.inputs etc
args = { 'val_train' : "train"
                      , 'n' : 10_000
                      , 'dist_type' : 'ints'
                      , 'm' : vocab_size
                      , 'std': 1.0
        }
teacher_slm.generate_data(**args)

In [ ]:
teacher_slm.model(teacher_slm.train_inputs).shape

In [ ]:
teacher_slm.train_inputs.shape

In [ ]:
teacher_slm.model.embedding.weight.detach().numpy()

In [ ]:
test = torch.argmax(teacher_slm.train_targets,axis = -1) #the code is write, i just need this to be a better dist.

In [ ]:
test

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
class_counts = [np.count_nonzero(test == i) for i in range(1, class_num + 1)]

# Create a bar chart
plt.bar(range(1, class_num + 1), class_counts, align='center')
plt.xlabel('Class')
plt.ylabel('Count')
plt.title('Bar Chart of Class Counts')
plt.xticks(range(1, class_num + 1))
plt.show()

specify gen outputs.  we need one hots
i can do this better than chat gpt lol

In [ ]:
##this function works, it is in LABNET.  
def one_hot_last_dim(tensor_shape):
    num_classes = tensor_shape[-1]
    random_idx = np.random.randint(1, num_classes + 1, size=tensor_shape[:-1])
    zero_tensor = np.zeros(tensor_shape, dtype=int)
    last_dim_indices = np.arange(num_classes)
    zero_tensor[..., :, last_dim_indices] = (random_idx[..., np.newaxis] == last_dim_indices)
    return zero_tensor